# Graduation Rate Data Exploration
In this notebook we look at elementary/secondary school data from the United States to hopefully shed some light on the challenges facing schools and school districts around the country. We are going to work through the entire data modeling pipeline from data acquisition -> data wrangling -> data exploration -> visualization -> modeling

### Questions
1. Where are schools struggling the most with low graduation rates?
2. Are these schools clustered in particular regions?
3. What demographic factors are correlated with low graduation rates?
4. Can we predict graduation rate and thereby reverse engineer some of the factors influencing graduation rates?

### Datasets
- <a href=https://nces.ed.gov/opengis/rest/services/K12_School_Locations/EDGE_GEOCODE_PUBLICSCH_1819/MapServer> Public School Locations </a>
- <a href=https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr> Graduation Rates </a> 
- <a href=https://data-nces.opendata.arcgis.com/datasets/nces::public-school-characteristics-2018-19/about> Demographics </a> 

### Skills In This Notebook
- REST queries
- Data Exploration
- Data Visualization
- Modeling/Prediction

***
## Imports
<code> pandas </code> data manipulation <br>
<code> plotly </code>   visualizations <br>
<code> requests </code>  manage our data scraping <br>
<code> dash </code>    interactive visualizations <br>
<code> scikitlearn </code> modeling and prediction

In [3]:
import pandas as pd
import plotly as plotly
import requests as reqs
import time
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import sklearn
import plotly.express as px
from requests.exceptions import HTTPError
from dash.dependencies import Input, Output

## Utility Functions
#### <code>reformat_grades(rate)</code>
removes non-numeric characters from our grades data

In [4]:
def reformat_grades(rate):
    if(rate.startswith('LE') or rate.startswith('LT') or rate.startswith('GE')):
        return rate[2:]
    if('-' in rate):
        return rate[0:rate.index('-')]
    return rate

## Data Files
<b> df_grad_rates </b> adjusted cohort graduation rates <br>
<b> df_locations </b> latitude and longitude for schools <br>
<b> df_state_abbrevs </b> mapping between state names and two letter abbreviations <br>
<b> df_demographics </b> school demographic and other characteristic data <br>

In [ ]:
df_grad_rates = pd.read_csv("../Data/18_19_Graduation_Rates.csv")
df_locations = pd.read_csv("../Data/school_geo_data.csv")
df_state_abbrevs = pd.read_csv("../Data/state_mapping.csv")
df_demographics = pd.read_csv("../Data/Public_School_Characteristics_2018-19.csv")

In [3]:
df_grad_rates = df_grad_rates[['STNAM','LEAID','SCHNAM','ALL_RATE_1819']] #only need state name, school district, school name, and graduation rate for now
df_grad_rates = df_grad_rates[df_grad_rates.ALL_RATE_1819 != 'PS'] #drop all rows for schools where data was protected

In [4]:
df_grad_rates['ALL_RATE_1819'] = df_grad_rates['ALL_RATE_1819'].apply(reformat_grades) #remove letters and symbols from graduation rates

In [5]:
#Replace state names with abbreviations
df_grad_rates['STNAM'] = df_grad_rates['STNAM'].str.lower()
df_state_abbrevs['Name'] = df_state_abbrevs['Name'].str.lower()
df_grad_rates = pd.merge(df_grad_rates,df_state_abbrevs, left_on='STNAM', right_on='Name')

In [6]:
#drop redundant columns after merge
df_grad_rates = df_grad_rates.drop(['STNAM','Name'], axis=1)
df_grad_rates.head()

,LEAID,SCHNAM,ALL_RATE_1819,Abbreviation
0,100005,Albertville High School,94,AL
1,100006,Asbury High School,90,AL
2,100006,Douglas High School,90,AL
3,100006,Kate D Smith DAR High School,95,AL
4,100006,Brindlee Mountain High School,85,AL


In [7]:
df_plot = pd.merge(df_grad_rates,df_locations,left_on=["SCHNAM","Abbreviation"],right_on=["NAME","STATE"])
df_plot["Grad_Rate"] = df_plot["ALL_RATE_1819"].astype('int32')

In [8]:
fig = px.scatter_geo(df_plot,lat="LAT",lon="LON", color="Grad_Rate", hover_name="SCHNAM", color_continuous_scale="inferno",scope='usa', width = 1200, height=800)

In [ ]:


app = dash.Dash()
app.layout = html.Div([
html.P("Filter by Graduation Rate"),
dcc.Slider(min=0, max=100,value=50,id="filter_slide",tooltip={"placement": "bottom", "always_visible": True}),
dcc.Graph(id="main_chart")
])

@app.callback(
    Output('main_chart', 'figure'),
    Input('filter_slide', 'value'))
def update_output(value):
    df = df_plot[df_plot.Grad_Rate < value]
    fig = px.scatter_geo(df,lat="LAT",lon="LON", color="Grad_Rate", hover_name="SCHNAM", color_continuous_scale="inferno",scope='usa',title="Graduation Rate By School", width = 1200, height=800)
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
